In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.tensorboard import SummaryWriter
import cv2
import numpy as np
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import random
from datetime import datetime

# Set paths
DATA_DIR = '/kaggle/input/kvasir-dataset/kvasir-instrument'
RESULTS_DIR = '/kaggle/working/results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# Set random seed for reproducibility
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything()

def get_preprocessing():
    return A.Compose([
        A.Resize(height=512, width=512),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

class KvasirSegmentationDataset(Dataset):
    def __init__(self, data_dir, split='train', transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.split = split
        
        # Get all image files
        image_dir = os.path.join(data_dir, 'images')
        mask_dir = os.path.join(data_dir, 'masks')
        
        if not os.path.exists(image_dir):
            raise RuntimeError(f'Image directory not found: {image_dir}')
        if not os.path.exists(mask_dir):
            raise RuntimeError(f'Mask directory not found: {mask_dir}')
        
        # Get all image files
        self.images = sorted([
            f for f in os.listdir(image_dir)
            if f.endswith('.jpg')
        ])
        
        # Verify corresponding masks exist
        valid_pairs = []
        for img_name in self.images:
            mask_name = img_name.replace('.jpg', '.png')
            mask_path = os.path.join(mask_dir, mask_name)
            
            if os.path.exists(mask_path):
                valid_pairs.append((img_name, mask_name))
            else:
                print(f"Warning: Mask not found for {img_name}, skipping...")
        
        if len(valid_pairs) == 0:
            raise RuntimeError(f'No valid image-mask pairs found in {data_dir}')
        
        print(f"Found {len(valid_pairs)} valid image-mask pairs")
        
        # Split dataset
        total_size = len(valid_pairs)
        train_size = int(0.8 * total_size)
        val_size = int(0.1 * total_size)
        
        if split == 'train':
            self.image_mask_pairs = valid_pairs[:train_size]
        elif split == 'val':
            self.image_mask_pairs = valid_pairs[train_size:train_size + val_size]
        else:  # test
            self.image_mask_pairs = valid_pairs[train_size + val_size:]
        
        print(f"{split.capitalize()} set size: {len(self.image_mask_pairs)}")
    
    def __len__(self):
        return len(self.image_mask_pairs)
    
    def __getitem__(self, idx):
        img_name, mask_name = self.image_mask_pairs[idx]
        img_path = os.path.join(self.data_dir, 'images', img_name)
        mask_path = os.path.join(self.data_dir, 'masks', mask_name)
        
        # Read image
        image = cv2.imread(img_path)
        if image is None:
            raise RuntimeError(f'Failed to load image: {img_path}')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Read mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise RuntimeError(f'Failed to load mask: {mask_path}')
        
        # Normalize mask to binary
        mask = (mask > 127).astype(np.float32)
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        
        # Add channel dimension to mask
        mask = mask.unsqueeze(0)
        
        return image, mask

def dice_loss(pred, target):
    smooth = 1.0
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum(dim=(2,3))
    union = pred.sum(dim=(2,3)) + target.sum(dim=(2,3))
    
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

def calculate_metrics(pred, target):
    pred = (pred > 0.5).float()
    target = target.float()
    
    # Calculate IoU
    intersection = (pred * target).sum()
    union = (pred + target).sum() - intersection
    iou = (intersection + 1e-8) / (union + 1e-8)
    
    # Calculate Dice
    dice = (2.0 * intersection + 1e-8) / (pred.sum() + target.sum() + 1e-8)
    
    return iou.item(), dice.item()

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    
    with tqdm(loader) as pbar:
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix({'loss': total_loss / (pbar.n + 1)})
    
    return total_loss / len(loader)

def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    dice_scores = []
    iou_scores = []
    
    with torch.no_grad():
        for images, masks in loader:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            
            # Calculate metrics
            iou, dice = calculate_metrics(outputs, masks)
            dice_scores.append(dice)
            iou_scores.append(iou)
            
            total_loss += loss.item()
    
    return total_loss / len(loader), np.mean(dice_scores), np.mean(iou_scores)

def plot_training_curves(train_losses, val_losses, dice_scores, iou_scores, save_dir):
    plt.figure(figsize=(15, 10))
    
    # Plot losses
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot Dice scores
    plt.subplot(2, 2, 2)
    plt.plot(dice_scores, label='Dice Score')
    plt.title('Validation Dice Score')
    plt.xlabel('Epoch')
    plt.ylabel('Dice Score')
    plt.legend()
    
    # Plot IoU scores
    plt.subplot(2, 2, 3)
    plt.plot(iou_scores, label='IoU Score')
    plt.title('Validation IoU Score')
    plt.xlabel('Epoch')
    plt.ylabel('IoU Score')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, 'training_curves.png'))
    plt.close()

def visualize_predictions(model, loader, device, save_dir, num_samples=5):
    model.eval()
    os.makedirs(os.path.join(save_dir, 'predictions'), exist_ok=True)
    
    with torch.no_grad():
        for i, (images, masks) in enumerate(loader):
            if i >= num_samples:
                break
                
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)['out']
            preds = (torch.sigmoid(outputs) > 0.5).float()
            
            for j in range(images.size(0)):
                plt.figure(figsize=(15, 5))
                
                # Original image
                plt.subplot(1, 3, 1)
                img = images[j].cpu().numpy().transpose(1, 2, 0)
                img = (img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])) * 255
                plt.imshow(img.astype(np.uint8))
                plt.title('Original Image')
                plt.axis('off')
                
                # Ground truth
                plt.subplot(1, 3, 2)
                plt.imshow(masks[j, 0].cpu().numpy(), cmap='gray')
                plt.title('Ground Truth')
                plt.axis('off')
                
                # Prediction
                plt.subplot(1, 3, 3)
                plt.imshow(preds[j, 0].cpu().numpy(), cmap='gray')
                plt.title('Prediction')
                plt.axis('off')
                
                plt.tight_layout()
                plt.savefig(os.path.join(save_dir, 'predictions', f'sample_{i}_{j}.png'))
                plt.close()

def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Create preprocessing
    preprocessing = get_preprocessing()
    
    # Create datasets
    train_dataset = KvasirSegmentationDataset(
        DATA_DIR,
        split='train',
        transform=preprocessing
    )
    val_dataset = KvasirSegmentationDataset(
        DATA_DIR,
        split='val',
        transform=preprocessing
    )
    test_dataset = KvasirSegmentationDataset(
        DATA_DIR,
        split='test',
        transform=preprocessing
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    
    # Create model
    model = deeplabv3_resnet50(num_classes=1).to(device)
    
    # Create optimizer and scheduler
    optimizer = optim.AdamW([
        {'params': model.backbone.parameters(), 'lr': 1e-5},
        {'params': model.classifier.parameters(), 'lr': 1e-4},
    ])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # Create criterion
    criterion = dice_loss
    
    # Initialize lists for tracking metrics
    train_losses = []
    val_losses = []
    dice_scores = []
    iou_scores = []
    
    # Create timestamped directory for results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    save_dir = os.path.join(RESULTS_DIR, f'deeplabv3_{timestamp}')
    os.makedirs(save_dir, exist_ok=True)
    
    # Training loop
    num_epochs = 50
    best_dice = 0.0
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        
        # Train
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        
        # Validate
        val_loss, dice_score, iou_score = validate(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        dice_scores.append(dice_score)
        iou_scores.append(iou_score)
        
        # Update scheduler
        scheduler.step(val_loss)
        
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Val Loss: {val_loss:.4f}")
        print(f"Dice Score: {dice_score:.4f}")
        print(f"IoU Score: {iou_score:.4f}")
        
        # Save best model
        if dice_score > best_dice:
            best_dice = dice_score
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_dice': best_dice,
            }, os.path.join(save_dir, 'best_model.pth'))
            print(f"Saved best model with Dice score: {best_dice:.4f}")
        
        # Plot training curves every 10 epochs
        if (epoch + 1) % 10 == 0:
            plot_training_curves(train_losses, val_losses, dice_scores, iou_scores, save_dir)
            visualize_predictions(model, val_loader, device, save_dir)
    
    # Final evaluation on test set
    test_loss, test_dice, test_iou = validate(model, test_loader, criterion, device)
    print(f"\nFinal Test Metrics:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Dice Score: {test_dice:.4f}")
    print(f"Test IoU Score: {test_iou:.4f}")
    
    # Save final metrics
    with open(os.path.join(save_dir, 'final_metrics.txt'), 'w') as f:
        f.write(f"Test Loss: {test_loss:.4f}\n")
        f.write(f"Test Dice Score: {test_dice:.4f}\n")
        f.write(f"Test IoU Score: {test_iou:.4f}\n")
    
    # Visualize test results
    visualize_predictions(model, test_loader, device, save_dir)
    
    print("Training completed!")

if __name__ == "__main__":
    main()

2025-04-25 21:43:00.599506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745617380.789735      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745617380.841736      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Using device: cuda
Found 590 valid image-mask pairs
Train set size: 472
Found 590 valid image-mask pairs
Val set size: 59
Found 590 valid image-mask pairs
Test set size: 59


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 215MB/s]
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Epoch 1/50


100%|██████████| 59/59 [04:03<00:00,  4.12s/it, loss=0.605]


...

Train Loss: 0.0190
Val Loss: 0.0505
Dice Score: 0.9438
IoU Score: 0.8958

Epoch 49/50


100%|██████████| 59/59 [04:14<00:00,  4.32s/it, loss=0.0188]


Train Loss: 0.0188
Val Loss: 0.0505
Dice Score: 0.9441
IoU Score: 0.8963

Epoch 50/50


100%|██████████| 59/59 [04:14<00:00,  4.31s/it, loss=0.0183]


Train Loss: 0.0183
Val Loss: 0.0504
Dice Score: 0.9435
IoU Score: 0.8949

Final Test Metrics:
Test Loss: 0.0444
Test Dice Score: 0.9589
Test IoU Score: 0.9219
Training completed!
